###  Mount Google Drive

In [ ]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3


Mounted at /content/drive


### Clone Codebase

In [ ]:
! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3
! cd /content/drive/MyDrive/COMP4901B-Homework3 && git clone https://github.com/hkust-nlp/COMP4901B-LLMs.git

### Download Dataset

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/setup.sh

### Q3 Start The Tuning

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3/ && bash scripts/self_train_gsm8k.sh